In [202]:
#pip install pyzx

In [203]:
import pyzx as zx

import sys, os, math
import random
import sympy as sym
from fractions import Fraction
from pyzx import print_matrix
from pyzx.basicrules import *

In [204]:
#--

In [205]:
strCirc = """
qreg q[8];
rz(1.25*pi) q[4];
rz(0.75*pi) q[5];

rz(0.25*pi) q[7];
cx q[5], q[7];

cx q[1], q[5];

cx q[4], q[5];
rz(0.75*pi) q[5];

rz(0.25*pi) q[7];
cx q[6], q[7];
rz(0.25*pi) q[7];
cx q[5], q[7];

cx q[4], q[5];
rz(0.75*pi) q[5];

rz(0.25*pi) q[7];
cx q[3], q[7];
rz(0.25*pi) q[7];
cx q[5], q[7];

cx q[4], q[5];
rz(0.75*pi) q[5];

rz(0.25*pi) q[7];
cx q[2], q[7];
rz(0.25*pi) q[7];
cx q[5], q[7];
rz(0.25*pi) q[7];
"""

c = zx.qasm(strCirc)
g = c.to_graph()
#g.normalize()
zx.draw(g, labels=True)
print("T-count = ", zx.tcount(g))

T-count =  13


In [206]:
xx = 2

zx.basicrules.fuse(g,23+xx,14+xx)
zx.basicrules.fuse(g,23+xx,8)
zx.basicrules.fuse(g,23+xx,32+xx)

zx.basicrules.fuse(g,12,9)
zx.basicrules.fuse(g,21+xx,15+xx)
zx.basicrules.fuse(g,30+xx,24+xx)
zx.basicrules.fuse(g,39+xx,33+xx)

zx.basicrules.fuse(g,13,15)

zx.draw(g, labels=True)
print("T-count = ", zx.tcount(g))
gBase = g.copy()

T-count =  13


In [207]:
#TEMP...
g.set_phase(12,0)
g.set_phase(23,0)
g.set_phase(32,0)
g.set_phase(41,0)
g.set_phase(25,0)
zx.draw(g, labels=True)

In [208]:
# This just re-numbers the vertex indexes to remove any gaps:
h = g.copy()
g = h.copy()
zx.draw(g, labels=True)

In [209]:
#NOTE:

# A limitation in this method is that you could have a node that has, say, 5 connections that each have just a 25% probability
# of being worthwhile (hence each of those contributing a weighting of 0.25). This node would then have a weighting of
# 0.25*5=1.25 (which will be read as 1 in the next tier up). So, this node is treated as a very likely worthwhile one, but it
# isn't really - it just has a lot of small probabilities adding up. Maybe we could take the highest child rather than summing
# them all in this way?

In [210]:
vweights = [0]*(max(g.vertices())+1)
vDoorsteps = [set() for i in range(max(g.vertices())+1)] # Doorstep nodes are T-gates on the doorstep to be cancelled if the block is removed
vCtrls = [set() for i in range(max(g.vertices())+1)] # This keeps track of the map for all red nodes of a CNOT -> their set of control nodes

for v in g.vertices(): # Be careful as g.vertices isn't normalised
    vweights[v] = 0
    #if g.phase(v) in (0.25, 0.75, 1.25, 1.75): vweights[v] = 1 # If the vertex is T-like, give it a base weighting of 1

for v in g.vertices():
    if g.type(v) == 2 and g.phase(v) in {0,1}: # If red vertex of n*pi-phase
        for neigh in g.neighbors(v):
            if g.qubit(neigh) != g.qubit(v): # If the neighbour is not on the same qubit (hence is a control)
                vCtrls[v].add(neigh) # Keep track of the CNOT's control spiders

for v in g.vertices():
    if g.type(v) == 2 and g.phase(v) in {0,1}: # If red vertex of n*pi-phase
        Tneighs = 0
        countedNeighs = set() # Keeps track of which T-gates have already been counted for this batch, to avoid overcounting
        for neigh in g.neighbors(v):
            if g.qubit(neigh) == g.qubit(v): # If the neighbour is on the same qubit
                if g.phase(neigh) in (0.25, 0.75, 1.25, 1.75): # If the neighbour also is T-like
                    if not(neigh in countedNeighs): # If the neighbour hasn't already been counted yet
                        Tneighs += 1
                        countedNeighs.add(neigh)

        #print()
        #print(v, "\t", Tneighs)
        #print(countedNeighs)

        if Tneighs == 2:
            w = 2/(len(vCtrls[v]))
            for ctrl in vCtrls[v]:

                isNew = True
                for i in countedNeighs:
                    if (i in vDoorsteps[ctrl]): isNew = False

                if isNew:
                    for i in countedNeighs: vDoorsteps[ctrl].add(i)
                    vweights[ctrl] += w

In [211]:
tier = 1
vtiers = newvweights = [0]*(max(g.vertices())+1)

# Weightings for lowest tier:
for i in range(len(vweights)):
    if (len(vDoorsteps[i]) > 0):
        print(i, "\t", vweights[i], "\t", vDoorsteps[i])
        vtiers[i] = tier
        
vweights_0 = vweights.copy()
vDoorsteps_0 = vDoorsteps.copy()

vweights_max = vweights.copy()
#vDoorsteps_max = vDoorsteps.copy()

10 	 2.0 	 {8, 13}
15 	 2.0 	 {16, 13}
18 	 2.0 	 {16, 21}
23 	 2.0 	 {24, 21}
26 	 2.0 	 {24, 28}
30 	 2.0 	 {28, 31}
33 	 2.0 	 {34, 31}


In [212]:
# Weightings as seen by next tier:
for i in range(len(vweights)):
    if (len(vDoorsteps[i]) > 0):
        w = min(vweights[i]/2,1.0)
        print(i, "\t", w, "\t", vDoorsteps[i])

10 	 1.0 	 {8, 13}
15 	 1.0 	 {16, 13}
18 	 1.0 	 {16, 21}
23 	 1.0 	 {24, 21}
26 	 1.0 	 {24, 28}
30 	 1.0 	 {28, 31}
33 	 1.0 	 {34, 31}


In [213]:
tier += 1
newvweights = [0]*(max(g.vertices())+1)
vDoorsteps = [set() for i in range(max(g.vertices())+1)] # Doorstep nodes are weighted nodes on the doorstep to be cancelled if the block is removed
vCtrls = [set() for i in range(max(g.vertices())+1)] # This keeps track of the map for all red nodes of a CNOT -> their set of control nodes

for v in g.vertices(): # Be careful as g.vertices isn't normalised
    vweights[v] = min(vweights_max[v]/2,1.0) # Update lower tiers' values for higher tier perspective
    newvweights[v] = 0
    #if g.phase(v) in (0.25, 0.75, 1.25, 1.75): vweights[v] = 1 # If the vertex is T-like, give it a base weighting of 1

for v in g.vertices():
    if g.type(v) == 2 and g.phase(v) in {0,1}: # If red vertex of n*pi-phase
        for neigh in g.neighbors(v):
            if g.qubit(neigh) != g.qubit(v): # If the neighbour is not on the same qubit (hence is a control)
                vCtrls[v].add(neigh) # Keep track of the CNOT's control spiders

for v in g.vertices():
    if g.type(v) == 2 and g.phase(v) in {0,1}: # If red vertex of n*pi-phase
        Tneighs = 0
        countedNeighs = set() # Keeps track of which weighted nodes have already been counted for this batch, to avoid overcounting
        for neigh in g.neighbors(v):
            if g.qubit(neigh) == g.qubit(v): # If the neighbour is on the same qubit
                if vweights[neigh] > 0: # If the neighbour also is weighted
                    if not(neigh in countedNeighs): # If the neighbour hasn't already been counted yet
                        Tneighs += 1
                        countedNeighs.add(neigh)

        #print()
        #print(v, "\t", Tneighs)
        #print(countedNeighs)

        if Tneighs == 2:
            w = 2/(len(vCtrls[v]))
            for ctrl in vCtrls[v]:

                isNew = True
                for i in countedNeighs:
                    if (i in vDoorsteps[ctrl]): isNew = False

                if isNew:
                    for i in countedNeighs: vDoorsteps[ctrl].add(i)
                    newvweights[ctrl] += w

In [214]:
for i in range(len(vweights)):
    vweights[i] = newvweights[i] # Update vweights from buffer
    
    if (vweights[i] > vweights_max[i]): vweights_max[i] = vweights[i] # Update max vals
    
    if (vweights[i]>0): #if (len(vDoorsteps[i]) > 0):
        print(i, "\t", vweights[i], "\t", vDoorsteps[i])
        vtiers[i] = tier
        
# Run the function above iteratively until no changes are made.
# It should also be buffered, so it doesn't read newly updated weights from this iteration.
# After all iterations then, we decomp the highest weighted node IN THE HIGHEST LAYER (i.e. the most levels above baseline)
# Then fuse and recalculate and repeat? -> May not actually even need to recalc and repeat, since we know the node's children
# Are we trying to maximise alpha at each step, or maximise the no. of total T-gates that will be removed?

12 	 1.0 	 {10, 18}
20 	 3.0 	 {26, 33, 10, 18}


In [215]:
#--

# Repeat the above function until no further changes occur?

#--

In [216]:
# Total weightings...

print("vertex\t weight\t tier")

for i in range(len(vweights)):
    if (vweights_max[i]>0): #if (len(vDoorsteps[i]) > 0):
        print(i, "\t", vweights_max[i], "\t", vtiers[i])

vertex	 weight	 tier
10 	 2.0 	 1
12 	 1.0 	 2
15 	 2.0 	 1
18 	 2.0 	 1
20 	 3.0 	 2
23 	 2.0 	 1
26 	 2.0 	 1
30 	 2.0 	 1
33 	 2.0 	 1


In [217]:
# Cut first the highest weighted of the highest tier

In [218]:
#TEMP...

def cut(v):
    x = g.row(v)
    y = g.qubit(v)

    for i in g.neighbors(v):
        newVert = g.add_vertex(2,y,x)
        g.add_edge((i,newVert), 1)

    g.remove_vertex(v)

    zx.draw(g, labels=True)
    #zx.simplify.full_reduce(g)
    #zx.draw(g, labels=True)

#----------

def cut(v):
    #for i in g.neighbors(v): g.add_to_phase(25,1,{}) # Add pi to each neighbour
    g.remove_vertex(v)
    
#----------

def apply_cut(v): # Return left and right branches of a vertex cut
    gLeft = g.copy()
    gRight = g.copy()
    gLeft.remove_vertex(v)
    gRight.remove_vertex(v)
    
    for i in g.neighbors(v):
        if g.type(i) == 2: # if red
            gRight.set_phase(i,gRight.phase(i)+1)
            
    return [gLeft,gRight]

#----------

def singleFusion(): # TEMP - this is a very inefficient method
    fusionFound = False
    for i in g.vertices():
        for j in g.neighbors(i):
            if g.type(i) > 0 and g.type(i) == g.type(j) and g.edge_type((i,j)) == 1:
                fusionFound = True
                break
        else: continue
        break
    if fusionFound:
        zx.basicrules.fuse(g,i,j)
    return fusionFound

def fullFusion():
    fullyFused = False
    while fullyFused == False:
        fullyFused = not singleFusion()

def idRemoval(): # TEMP - currently also removes pi-phase 2-edged spiders (for simplicity)
    deadSpiders = set() # Spiders marked for removal

    for i in g.vertices():
        if g.phase(i) in {0,1} and len(g.neighbors(i)) == 2: # Identity removal (and, TEMP, pi-phase removal - for simplicity)
            deadSpiders.add(i)

    for i in deadSpiders:
        neighs = list()
        for neigh in g.neighbors(i): # necessarily only has 2 neighbours here
            neighs.append(neigh)

        g.add_edge((neighs[0],neighs[1]),1)
        g.remove_vertex(i)

In [219]:
cutChoice = 20 # the choice of vertex to cut

zx.draw(g, labels=True, scale=20)
gList = apply_cut(cutChoice)
zx.draw(g, labels=True, scale=20)

In [220]:
#TEMP:
zx.draw(gList[0],labels=True,scale=20)
zx.draw(gList[1],labels=True,scale=20)

In [221]:
#TEMP:
g = gList[1].copy()

In [222]:
fullFusion()
idRemoval()
fullFusion()
zx.draw(g, labels=True)

# TODO - this should be flipping the phase of #26 and adding a pi to #25

In [223]:
#--

# Benchmarking...

#--

In [262]:
strCirc = """
qreg q[8];
ccz q[0], q[1], q[3];
ccz q[0], q[1], q[6];
"""

c = zx.qasm(strCirc)
g = c.to_graph()
#g.normalize()
zx.draw(g, labels=True, scale=20)
print("T-count = ", zx.tcount(g))

T-count =  14


In [288]:
zx.simplify.full_reduce(g)
zx.draw(g, labels=True, scale=20)
print("T-count = ", zx.tcount(g))

T-count =  12


In [289]:
gDecomp = zx.simulate.find_stabilizer_decomp(g)
print("No. of terms:",len(gDecomp))

No. of terms: 33


In [290]:
alpha = 0.47 # the alpha value of the decomposition (e.g. BSS = 0.47)

#zx.simplify.full_reduce(g)
#zx.draw(g, labels=True, scale=20)
print("T-count:", zx.tcount(g))
print("Upper bound estimate of no. of terms:", 2**(alpha*zx.tcount(g)))

# Note: this "upper bound" estimate can be lower than the actual no. of terms if the T-count is very low
# (since then the lower alpha decomps for the remaining sets of T-gates, when fewer than 7 remain, are more significant)

T-count: 12
Upper bound estimate of no. of terms: 49.86653309827198


In [267]:
#--

In [291]:
g.apply_state("++-+-+--")
g.apply_effect("-++---++")
zx.draw(g, scale=20)
print("T-count:", zx.tcount(g))

In [292]:
zx.simplify.full_reduce(g)
zx.draw(g, scale=20)
print("T-count:", zx.tcount(g))
print("Upper bound estimate of no. of terms:", 2**(alpha*zx.tcount(g)))

T-count: 12
Upper bound estimate of no. of terms: 49.86653309827198


In [293]:
gDecomp = zx.simulate.find_stabilizer_decomp(g)
print("No. of terms:",len(gDecomp))

No. of terms: 0


In [231]:
# TODO - WHY IS THIS COMING OUT AS ZERO??

In [232]:
#





#TEMP...

In [287]:
strCirc = """
qreg q[8];
ccz q[5], q[6], q[7];
cx q[4], q[5];
cx q[3], q[6];
ccz q[5], q[6], q[7];
"""

c = zx.qasm(strCirc)
g = c.to_graph()
#g.normalize()
zx.draw(g, labels=True)
print("T-count = ", zx.tcount(g))

T-count =  14
